In [65]:
import pandas as pd
import re
from string import punctuation
punct = punctuation+'«»—…“”*№–'
import itertools
regex = re.compile("[А-ЯЙ]{3,}") 

## 1. Читаем 2 датасета и объединяем в 1 датафрейм

In [66]:
df1 = pd.read_csv("capitalization-vdud-wall-2.csv")
df2 = pd.read_csv("capitalization-echo-wall-2.csv")

In [68]:
df1.head()

,from_id,post_id,comment_text,comment_id,post_text,first_name,last_name,sex,bdate,city
0,19670600.0,13.0,"Трек должен называться ""Тем кто знает АУЕ"" )",1651,[id2128351|Баста] - о Немагии и своей зарплате...,Vladimir,Kolyaskin,2,31.10,Port-au-Prince
1,1625187.0,13.0,"трек ""РУСЗМ"" ( родина умыта слезами зеков и ма...",396,[id2128351|Баста] - о Немагии и своей зарплате...,Sergey,Kostyrev,2,30.1.1987,Blagoveschensk
2,49407875.0,22.0,ДНЕВНИК ХАЧА,247,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Timofey,Ezhov,2,14.10,Moscow
3,423900905.0,22.0,"Планируется открытие нового канала "" Журнал ЧУ...",2768,[id8658426|Амиран Сардаров] (Дневник Хача) - с...,Svetlana,Mironova,1,6.5.1994,Moscow
4,312160075.0,28.0,"Как же жалок Соболев, на фоне более сильной ли...",2471,"[id2183360|Соболев] - о Путине, митингах и сек...",Anastasia,Smirnova,1,27.10.1997,Obninsk


In [69]:
df1 = df1.drop(["post_text"], axis = 1)

In [70]:
df1.tail()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city
2768,11711559.0,174960.0,"Мне жууутко понравилось. Хорошие мысли, слова;...",175375,Ekaterina,Pozharova,1,30.7.1989,Samara
2769,138291595.0,174960.0,#конкурс Дисклеймер: канал «вДудь» не пропаган...,175427,Darya,Golovanova,1,7.12,Tver
2770,17779627.0,174960.0,"#конкурс\n""ЧЕЛОВЕК ЛЕСА"" (2013) by William Dou...",175477,Maria,Bogachek,1,21.8,Saint Petersburg
2771,181037157.0,174960.0,"#конкурс\nМастер 2012 год, режиссера Пола Тома...",175168,Daniil,Markeev,2,7.9.2000,Ivanovo
2772,153266583.0,174960.0,ПОБЕДА!\n\nhttps://tvrain.ru/news/mvd_prekrati...,175049,Matrena,Sidorova,1,NaN,Cheboxary


In [71]:
df2.tail()

,from_id,post_id,comment_text,id,first_name,last_name,sex,bdate,city
2416,167423223.0,3173969.0,ВОРобьев? 😊,3174084,Andrey,Motovilov,2,21.9,Tolyatti
2417,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД...там о...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow
2418,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Австрии.",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod
2419,209279951.0,3173952.0,"[id289354141|Светлана], ответ один: НИКОГДА!!!",3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk
2420,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh


In [72]:
df2.columns = ['from_id', 'post_id', 'comment_text', "comment_id", "first_name", "last_name", "sex", "bdate", "city"]

In [73]:
df = pd.concat([df1, df2], axis=0)

In [74]:
df.reset_index(drop=True, inplace=True)

In [75]:
df.sex.value_counts() # изначально 4 тыс. комментов мужского пола, 973 - женского.  

2    4220
1     973
0       1
Name: sex, dtype: int64

In [76]:
df.tail()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city
5189,167423223.0,3173969.0,ВОРобьев? 😊,3174084,Andrey,Motovilov,2,21.9,Tolyatti
5190,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД...там о...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow
5191,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Австрии.",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod
5192,209279951.0,3173952.0,"[id289354141|Светлана], ответ один: НИКОГДА!!!",3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk
5193,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh


## 2. Немного очищаем тексты комментариев (избавляем от ссылок на юзеров, хэштегов)

In [77]:
# удалить записи в квадратных скобках
for i in df.index:
    if re.search("\[id.*|.*\],", df.iloc[i, 2]):
        df.iloc[i, 2] = re.sub(r"(\[id.*?|.*?\]), ", r'', df.iloc[i, 2])

In [78]:
# удалить записи в квадратных скобках
for i in df.index:
    if re.search("\[club.*|.*\],", df.iloc[i, 2]):
        df.iloc[i, 2] = re.sub(r"(\[club.*?|.*?\]), ", r'', df.iloc[i, 2])

In [79]:
# удалить хэштеги
for i in df.index:
    if re.search("(#.*?) ", df.iloc[i, 2]):
        df.iloc[i, 2] = re.sub("(#.*?) ", '', df.iloc[i, 2])

In [80]:
df.tail()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city
5189,167423223.0,3173969.0,ВОРобьев? 😊,3174084,Andrey,Motovilov,2,21.9,Tolyatti
5190,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД...там о...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow
5191,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Австрии.",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod
5192,209279951.0,3173952.0,ответ один: НИКОГДА!!!,3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk
5193,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh


## 3. Добавляем в отдельную колонку число токенов в комменте

In [81]:
df["n_tokens"] = None

In [82]:
# токенизировать и добавить в колонку число слов в комменте
for i in df.index:
    df.iloc[i, 2] = df.iloc[i, 2].replace(".", ". ")
    df.iloc[i, 2] = df.iloc[i, 2].replace(",", ", ")
    df.iloc[i, 2] = df.iloc[i, 2].replace("!", "! ")
    df.iloc[i, 2] = df.iloc[i, 2].replace("?", "? ")
    df.iloc[i, 2] = df.iloc[i, 2].replace("(", "( ")
    df.iloc[i, 2] = df.iloc[i, 2].replace(")", ") ")
    text = df.iloc[i, 2]
    tokens = [i.strip(punct) for i in text.split() if i.strip(punct)]
    df.iloc[i, -1] = len(tokens)

In [83]:
df.tail(5)

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city,n_tokens
5189,167423223.0,3173969.0,ВОРобьев? 😊,3174084,Andrey,Motovilov,2,21.9,Tolyatti,2
5190,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД. . . та...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow,36
5191,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Авст...",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod,10
5192,209279951.0,3173952.0,ответ один: НИКОГДА! ! !,3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk,3
5193,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh,2


## 4. Также добавляем колонки с числом слов, написанных капсом, и самими словами

In [84]:
df["n_caps"] = None
df["caps"] = None

In [85]:
# добавить в колонку число слов, написанных капсом; сами эти слова
for i in df.index:
    text = df.iloc[i, 2]
    tokens = [i.strip(punct) for i in text.split() if i.strip(punct)]
    caps = [w for w in tokens if w.isupper() and len(w) > 2 and re.search(regex, w)]
    df.iloc[i, -1] = ' '.join(caps)
    df.iloc[i, -2] = len(caps)

In [86]:
df.tail()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city,n_tokens,n_caps,caps
5189,167423223.0,3173969.0,ВОРобьев? 😊,3174084,Andrey,Motovilov,2,21.9,Tolyatti,2,0,
5190,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД. . . та...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow,36,1,МВД
5191,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Авст...",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod,10,1,США
5192,209279951.0,3173952.0,ответ один: НИКОГДА! ! !,3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk,3,1,НИКОГДА
5193,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh,2,2,НЕТ СТРАНЫ


## 5. Удаляем посты, где не оказалось ни одного капса

In [87]:
# удаляем посты, где не оказалось ни одного капса
df = df[df.n_caps != 0]
df.reset_index(drop=True, inplace=True)
df.sex.value_counts()

2    3947
1     925
0       1
Name: sex, dtype: int64

In [88]:
df = df[df.sex != 0]
df.reset_index(drop=True, inplace=True)
df.sex.value_counts()

2    3947
1     925
Name: sex, dtype: int64

## 6. Для того, чтобы анализировать не просто все, а только "осознанные" капслоки, создадим еще один дата-фрейм

In [89]:
df_filtered = df.iloc[:, :-2]

In [90]:
df_filtered.head()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city,n_tokens
0,19670600.0,13.0,"Трек должен называться ""Тем кто знает АУЕ"" )",1651,Vladimir,Kolyaskin,2,31.10,Port-au-Prince,7
1,1625187.0,13.0,"трек ""РУСЗМ"" ( родина умыта слезами зеков и м...",396,Sergey,Kostyrev,2,30.1.1987,Blagoveschensk,8
2,49407875.0,22.0,ДНЕВНИК ХАЧА,247,Timofey,Ezhov,2,14.10,Moscow,2
3,423900905.0,22.0,"Планируется открытие нового канала "" Журнал ЧУ...",2768,Svetlana,Mironova,1,6.5.1994,Moscow,6
4,312160075.0,28.0,"Как же жалок Соболев, на фоне более сильной л...",2471,Anastasia,Smirnova,1,27.10.1997,Obninsk,36


## 7. Создадим список всех токенов и рассчитаем для каждого слова, написанного капслоком, метрику, на основе нашего маленького корпуса: 
_слово написано капсом / (слово написано капсом + слово написано не капсом)_

In [91]:
all_text = [df.iloc[i, 2] for i in df.index]
all_tokens = [[i.strip(punct) for i in text.split() if i.strip(punct)] for text in all_text]
all_tokens_flat = list(itertools.chain.from_iterable(all_tokens))
caps = [i for i in all_tokens_flat if i.isupper()]

In [92]:
len(all_tokens_flat)

186640

In [28]:
caps_dict = {i:[] for i in caps}

In [29]:
for word in caps_dict.keys():
    upper_count = all_tokens_flat.count(word)
    lower_count = all_tokens_flat.count(word.lower())
    total_count = upper_count + lower_count
    metrics = upper_count / total_count
    caps_dict[word] = [metrics, total_count]

In [30]:
caps_dict["ЛЫСЫЙ"]

[1.0, 15]

Это означает, что слово "ЛЫСЫЙ" встречается в корпусе 15 раз и все разы - с капсом. Странно, правда? Наверное, у меня очень маленький корпус.

In [31]:
caps_dict["АНТИХАЙП"]

[0.8888888888888888, 9]

И антихайп...

In [32]:
caps_dict["ИМХО"]

[0.9142857142857143, 35]

А вот это ожидаемо. 

## 8. Нужно как-то отсеять слова, которые осознанно пишутся капслоком. Я взяла высокий порог 0.85 (поскольку маленький корпус) и затем отредактировала списки слов глазами, рукодоствуясь здравым смыслом. 

In [33]:
meaningful_caps = []
usual_caps = []
the_rest = []

for k, v in caps_dict.items():
    if v[1] > 4 and v[0] < 0.85:
        meaningful_caps.append(k)
    elif v[1] > 4 and v[0] > 0.85:
        usual_caps.append(k)
    else:
        the_rest.append(k)

In [34]:
with open("meaningful_caps.txt", "w", encoding = "UTF-8") as f:
    for i in meaningful_caps:
        f.write(i + "\n")

In [35]:
with open("usual_caps.txt", "w", encoding = "UTF-8") as f:
    for i in usual_caps:
        f.write(i + "\n")

In [36]:
with open("the_rest.txt", "w", encoding = "UTF-8") as f:
    for i in the_rest:
        f.write(i + "\n")

## 9. Читаем получившийся файл и используем его, чтобы создать колонки в датафрейме, аналогичные тому, что выше, но только для "осознанного" капслока

In [93]:
with open("meaningful_caps.txt", "r", encoding = "UTF-8") as f:
    f = f.readlines()
meaningful_caps = [i.strip() for i in f]
meaningful_caps = [i for i in meaningful_caps if len(i) > 2 and re.search(regex, i)]
meaningful_caps = list(set(meaningful_caps))

In [94]:
len(meaningful_caps)

3923

In [95]:
meaningful_caps[:10]

['НЕВЗОРИШКА',
 'НИХУЯ',
 'БОГ',
 'КРИД',
 'ЕВРО',
 'КАМЕДИИ',
 'ДОКОЛЕ',
 'АЙФОНЧИК',
 'КАЙФ',
 'ЧУДЕСНАЯ']

In [97]:
# вернемся к датафрейму
df_filtered.tail()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city,n_tokens
4867,307041317.0,3173979.0,конечно! Очень выгодно взять деньги из эконом...,3174194,Evgeny,Printsev,2,29.7.1972,Kemerovo,18
4868,486523749.0,3173952.0,Отвалите от сына главного оборотня МВД. . . та...,3173974,Alexey,Vitoshnov,2,6.6.1986,Moscow,36
4869,316971518.0,3173952.0,"У Собянина сыновья живут в США, а дочь в Авст...",3174491,Nina,Kondratyeva,1,17.12.1954,Nizhny Novgorod,10
4870,209279951.0,3173952.0,ответ один: НИКОГДА! ! !,3174770,Amiran,Kravishvili,2,24.9.1954,Pyatigorsk,3
4871,110840513.0,3173883.0,НЕТ СТРАНЫ.,3173910,Mikhail,Dobromyslov,2,13.9.1960,Pudozh,2


In [98]:
df_filtered["n_caps"] = None
df_filtered["caps"] = None

In [99]:
# добавить в колонку число слов, написанных капсом; сами эти слова
for i in df_filtered.index:
    text = df_filtered.iloc[i, 2]
    tokens = [i.strip(punct) for i in text.split() if i.strip(punct)]
    caps = [w for w in tokens if w.isupper() and w in meaningful_caps]
    df_filtered.iloc[i, -1] = ' '.join(caps)
    df_filtered.iloc[i, -2] = len(caps)

In [100]:
df_filtered.head()

,from_id,post_id,comment_text,comment_id,first_name,last_name,sex,bdate,city,n_tokens,n_caps,caps
0,19670600.0,13.0,"Трек должен называться ""Тем кто знает АУЕ"" )",1651,Vladimir,Kolyaskin,2,31.10,Port-au-Prince,7,0,
1,1625187.0,13.0,"трек ""РУСЗМ"" ( родина умыта слезами зеков и м...",396,Sergey,Kostyrev,2,30.1.1987,Blagoveschensk,8,1,РУСЗМ
2,49407875.0,22.0,ДНЕВНИК ХАЧА,247,Timofey,Ezhov,2,14.10,Moscow,2,2,ДНЕВНИК ХАЧА
3,423900905.0,22.0,"Планируется открытие нового канала "" Журнал ЧУ...",2768,Svetlana,Mironova,1,6.5.1994,Moscow,6,1,ЧУРКИ
4,312160075.0,28.0,"Как же жалок Соболев, на фоне более сильной л...",2471,Anastasia,Smirnova,1,27.10.1997,Obninsk,36,1,КАЖДОМУ


In [101]:
df_filtered.comment_text[2]

'ДНЕВНИК ХАЧА'

In [102]:
df_filtered = df_filtered[df_filtered.n_caps != 0]
df_filtered.reset_index(drop=True, inplace=True)
df_filtered.sex.value_counts()

2    2044
1     522
Name: sex, dtype: int64

Получилось почти в 2 раза меньше наблюдений. 

## 10. Сохраняем итоговые результаты в файлы

In [103]:
df.to_csv("capitalization.csv", index=False)
df_filtered.to_csv("meaningful-capitalization.csv", index=False)